In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
data = np.load('P-1.npy')
data

In [ ]:
np.shape(data)

In [ ]:
np.isnan(data).sum()

In [ ]:
data[0]

In [ ]:
data[10:20]

In [ ]:
np.diff(data)

array([[0.69516194, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.68570386, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.72571851, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.31611504, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.2971992 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.73226619, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(data)
plt.xlabel('Time step')
plt.ylabel('Value')
plt.title('Time Series Data over Time steps')
plt.show()

In [ ]:
window_size = 250
X = []
y = []
for i in range(len(data)):
  last_idx = i + window_size
  if last_idx > len(data)-1:
    break
  X.append(data[i:last_idx])
  y.append(data[last_idx])
X = np.array(X)
y = np.array(y)


In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
model = Sequential()
model.add(LSTM(units = 80, return_sequences=True, input_shape = (250, 25)))
model.add(Dropout(0.3))
model.add(LSTM(units=80))
model.add(Dropout(0.3))
model.add(Dense(1))

model.compile(optimizer='adam', loss= 'mean_squared_error')

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
history = model.fit(X,y, epochs=100, batch_size = 64, validation_split=0.1, callbacks=[early_stopping])


In [ ]:
test_data = np.load('P-1_test.npy')
test_data.shape

In [ ]:
window_size = 250
X_test = []
y_test = []
for i in range(len(test_data)):
  last_idx = i + window_size
  if last_idx > len(test_data)-1:
    break
  X_test.append(data[i:last_idx])
  y_test.append(data[last_idx])
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
h = 2100
h_err = []
h_win = []
for i in range(len(y_test)):
  y_pred = model.predict(X_test[i:i+1])
  err = np.abs(y_test[i] - y_pred[0,0])

  h_err.append(err)

  if len(h_err) > h:
    h_err.pop(0)
  h_win.append(h_err.copy())

for i, j in enumerate(h_win):
  print(f'time={i}, historical_errror_window = {j}')


In [ ]:
alpha = 0.005
smoothed_err = []
sm_err_prev = None

for i in h-err:

  if sm_err_prev is None:
    err_sm = i
  else:
    err_sm = alpha * i + (1-alpha)* sm_err_prev

  sm_err_prev = err_sm

  smoothed_err.append(err_sm)

  print(f'Error window: {smoothed_err}, EWMA: {err_sm}')


In [ ]:
len(smoothed_err)

In [ ]:
mu = np.mean(smoothed_err)
sigma = np.std(smoothed_err)

opt_epsilon = None
opt_score = -np.inf

z = [2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10]

for i in z:
  epsilon = mu + i * sigma

  for idx in smoothed_err:
    eseq = []
    ea = []
    normal=[]
    for j in idx:
      if j > epsilon:
        ea.append(j)
        break
      else:
        normal.append(j)

    if  len(ea) > 0:
      eseq.append(j)
  seq_len = len(eseq)
  print("Seq length:", seq_len)

  del_mu = mu - np.mean(normal)
  del_sigma = sigma - np.std(normal)


  try:
    score = (del_mu / mu + del_sigma / sigma) / (len(ea) * seq_len**2)
  except (ZeroDivisionError, ValueError) as e:
    print("Error", e)

  if score > opt_score:
    opt_score = score
    opt_epsilon = epsilon
    Eseq = eseq
    Ea = ea

In [ ]:
opt_epsilon

In [ ]:
for i in Eseq:
  print(i)

In [ ]:
for i in eseq:
  print(i)

In [ ]:
len(Eseq)

In [ ]:
e_max = []
for i in eseq:
  max_e = max(i)
  e_max.append(max_e)
  s = (e_max - opt_epsilon)/ (mu + sigma)

In [ ]:
normal_max = max(normal)
normal_max

np.float64(0.9485863596200943)

In [ ]:
e_max.append(normal_max)